# Bootstrapping a Linear Regression

This notebook implements both a traditional bootstrap and a block bootstrap - in order to get more robust standard errors of OLS coefficients.

## Load Packages and Extra Functions

In [1]:
using Dates, DelimitedFiles, Statistics, LinearAlgebra, Random

include("jlFiles/printmat.jl")
include("jlFiles/OlsFn.jl")
include("jlFiles/NWFn.jl")

NWFn

## Loading Data

The regressions used below are of the type

$
y_t = x_t'b + u_t
$

where $y_t$ are monthly data on 1-year excess returns on a bond and $x_t$ are lagged (12 months) forward rates. 

In [2]:
xx  = readdlm("Data/BondPremiaPs.csv",',',skipstart=1)
rx  = xx[:,5]                     #bond excess returns
f   = xx[:,6:end]                 #forward rates

x = [ones(size(f,1)-12) f[1:end-12,:]]   #regressors
y = rx[13:end]                           #dependent variable


(T,n) = (size(y,1),size(y,2))            #no. obs and no. test assets
K     = size(x,2)                        #no. regressors

println("T = $T, n = $n, K = $K")

T = 580, n = 1, K = 6


## Point Estimates

In [3]:
(bLS,u,yhat,Covb,) = OlsGMFn(y,x)            #OLS estimate and traditional std errors
StdbLS = sqrt.(diag(Covb))                   #Covb is Cov(sqrt(T)b)

printblue("OLS estimates:")
rowNames = [string("x",i) for i=1:K]
printTable([bLS  StdbLS],["coeff","std (trad.)"],rowNames,width=15)

OLS estimates:
            coeff    std (trad.)
x1         -3.306          0.824
x2         -4.209          0.712
x3         10.627          4.513
x4        -14.397         12.896
x5          7.096         15.876
x6          1.284          6.904



## Standard Bootstrap (I)

In each loop, a new series of residuals, $\tilde{u}_{t}$, is created by drawing (with replacement) values from the fitted residuals. Then, simulated values of the dependent variable are created as $\tilde{y}_{t}=x_{t}^{\prime}\beta+\tilde{u}_{t}$ and we redo the estimation on ($\tilde{y}_{t},x_{t}$).

This is repeated `NSim` times.

In [4]:
NSim      = 2000                 #no. of simulations
Random.seed!(123)

bBoot   = fill(NaN,(NSim,K))
for i = 1:NSim                                       #loop over simulations
  #local t_i, vv_i, utilde, ytilde, b_i              #only needed in script
  t_i        = rand(1:T,T)                           #T random numbers from 1:T (with replacement)
  #println(t_i)                                      #uncomment to see which rows that are picked out
  utilde     = u[t_i]
  ytilde     = x*bLS + utilde[1:T]
  bBoot[i,:] = OlsGMFn(ytilde,x)[1]
end

printblue("Coefficients:")
xx = [bLS  mean(bBoot,dims=1)']
printTable(xx,["OLS","avg. bootstr"],rowNames,width=20)

printblue("Std:")
xx = [StdbLS std(bBoot,dims=1)']
printTable(xx,["trad.","bootstrap 1"],rowNames,width=20)

printred("looks like no particular need for bootstrap in this case, but...see below")

Coefficients:
                   OLS        avg. bootstr
x1              -3.306              -3.294
x2              -4.209              -4.218
x3              10.627              10.647
x4             -14.397             -14.303
x5               7.096               6.897
x6               1.284               1.379

Std:
                 trad.         bootstrap 1
x1               0.824               0.832
x2               0.712               0.704
x3               4.513               4.511
x4              12.896              12.948
x5              15.876              16.033
x6               6.904               7.010

looks like no particular need for bootstrap in this case, but...see below


## Block Bootstrap (II)

To handle autocorrelated residuals, we now consider a *block bootstrap*.


In each loop, we initially define a random starting point (row number) of each block (by using the `rand()` function)---and create a vector of all rows that are in a block. For instance, suppose we randomly draw that the blocks should start on rows $27$ and $35$ and that we have decided that each block should contain $10$ rows, then the artificial sample will pick out rows $27-36$ and $35-44$. Clearly, some rows can be in several blocks. Once we have $T$ rows, we define a new series of residuals, $\tilde{u}_{t}$.

Then, new values of the dependent variable are created as $\tilde{y}_{t}=x_{t}^{\prime}\beta+\tilde{u}_{t}$ and we redo the estimation on ($\tilde{y}_{t},x_{t}$).

In [5]:
BlockSize = 10                   #size of blocks
NSim      = 2000                 #no. of simulations
Random.seed!(123)

nBlocks = round(Int,ceil(T/BlockSize))             #number of blocks, rounded up
bBoot2  = fill(NaN,(NSim,K*n))                       #vec(b), [beq1 beq2..beqn]
for i = 1:NSim                                       #loop over simulations
  #local t_i, vv_i, utilde, ytilde, b_i              #only needed in script
  t_i         = rand(1:T,nBlocks,1)                   #nBlocks x 1, random starting row of blocks
  t_i         = t_i .+ collect(0:BlockSize-1)'        #nBlocks x BlockSize, each row is a block
  vv_i        = t_i .> T
  t_i[vv_i]   = t_i[vv_i] .- T                        #wrap around if index > T
  #println(t_i)                                       #uncomment to see which rows that are picked out
  t_i         = vec(t_i')                             #column vector of the blocks
  utilde      = u[t_i,:]
  ytilde      = x*bLS + utilde[1:T,:]
  bBoot2[i,:] = OlsGMFn(ytilde,x)[1]
end

printblue("Std:")
xx = [StdbLS std(bBoot,dims=1)' std(bBoot2,dims=1)']
printTable(xx,["trad.","bootstrap 1","block bootstr"],rowNames,width=20)

printred("this bootstrap handles autocorrelation, so the std tend to be higher")

Std:
                 trad.         bootstrap 1       block bootstr
x1               0.824               0.832               2.099
x2               0.712               0.704               1.398
x3               4.513               4.511               8.054
x4              12.896              12.948              23.102
x5              15.876              16.033              28.880
x6               6.904               7.010              12.809

this bootstrap handles autocorrelation, so the std tend to be higher
